# 데이터 수집 시작!

In [1]:
import json
import re
import nltk
import pandas as pd
import numpy as np
import time
import requests

from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from konlpy.tag import Twitter
from nltk.corpus import stopwords
from collections import Counter

In [2]:
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 '}

## url 추출

In [3]:
driver = webdriver.Chrome(executable_path='(driver) chromedriver.exe')

In [4]:
url_list=[]

In [5]:
page = 1

while page<=28:
    try:
        url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&fltDispCatNo=&prdSort=01&pageIdx='+str(page)+'&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010008_Small'
        driver.get(url)
        #button = driver.find_element_by_xpath('/html/body/div[3]/div[8]/div[1]/div[5]/div[2]/ul/li[3]/a')
        #button.click()
        web_page = BeautifulSoup(driver.page_source,'html.parser')
        for urls in web_page.find_all('a',{'class':'prd_thumb goodsList'}):
            url_list.append(urls['href'])
        print(str(page)+"번째 페이지 완료")
        page+=1
    except:
        print("오류 발생")
        break

1번째 페이지 완료
2번째 페이지 완료
3번째 페이지 완료
4번째 페이지 완료
5번째 페이지 완료
6번째 페이지 완료
7번째 페이지 완료
8번째 페이지 완료
9번째 페이지 완료
10번째 페이지 완료
11번째 페이지 완료
12번째 페이지 완료
13번째 페이지 완료
14번째 페이지 완료
15번째 페이지 완료
16번째 페이지 완료
17번째 페이지 완료
18번째 페이지 완료
19번째 페이지 완료
20번째 페이지 완료
21번째 페이지 완료
22번째 페이지 완료
23번째 페이지 완료
24번째 페이지 완료
25번째 페이지 완료
26번째 페이지 완료
27번째 페이지 완료
28번째 페이지 완료


## 제품코드, url 링크 끌어오기

In [6]:
# 제품코드 끌어오기
product = []

for i in range(1,29):
    url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&fltDispCatNo=&prdSort=01&pageIdx='+str(i)+'&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010008_Small'
    web = urlopen(url)
    web_page = BeautifulSoup(web,'html.parser')
    product += web_page.find_all('a',{'class':'prd_thumb goodsList'})

product_num=[]

for i in range(len(product)):
    product_num.append(product[i]['data-ref-goodsno'])

## 제이슨파일로 상품 관련 자료 끌어오기

In [7]:
## 제품명, 브랜드 끌어오기
product_name = []
product_brand = []
for i in range(1,29):
    url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&fltDispCatNo=&prdSort=01&pageIdx='+str(i)+'&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010008_Small'
    web = urlopen(url)
    web_page = BeautifulSoup(web,'html.parser')
    product_name += web_page.find_all('p',{'class':'tx_name'})
    product_brand += web_page.find_all('span',{'class':'tx_brand'})
    
product_name_real=[]
product_brand_real=[]

for i in range(len(product_name)):
    product_name_real.append(product_name[i].get_text())
    product_brand_real.append(product_brand[i].get_text())

In [9]:
# 상품별 전체 리뷰 수 끌어오기
product_link=[]

for i in range(len(product)):
    product_link.append(product[i]['href'])

product_total_review = []
check = 1

for i in product_link:
    web = requests.get(i, headers=headers).text
    web_page = BeautifulSoup(web,'html.parser')
    review_count = web_page.find('a',{'class':'goods_reputation'})
    review_count = review_count.find('span').get_text()
    product_total_review.append(review_count[1:len(review_count)-1])
    print(check,"완료")
    check+=1

1 완료
2 완료
3 완료
4 완료
5 완료
6 완료
7 완료
8 완료
9 완료
10 완료
11 완료
12 완료
13 완료
14 완료
15 완료
16 완료
17 완료
18 완료
19 완료
20 완료
21 완료
22 완료
23 완료
24 완료
25 완료
26 완료
27 완료
28 완료
29 완료
30 완료
31 완료
32 완료
33 완료
34 완료
35 완료
36 완료
37 완료
38 완료
39 완료
40 완료
41 완료
42 완료
43 완료
44 완료
45 완료
46 완료
47 완료
48 완료
49 완료
50 완료
51 완료
52 완료
53 완료
54 완료
55 완료
56 완료
57 완료
58 완료
59 완료
60 완료
61 완료
62 완료
63 완료
64 완료
65 완료
66 완료
67 완료
68 완료
69 완료
70 완료
71 완료
72 완료
73 완료
74 완료
75 완료
76 완료
77 완료
78 완료
79 완료
80 완료
81 완료
82 완료
83 완료
84 완료
85 완료
86 완료
87 완료
88 완료
89 완료
90 완료
91 완료
92 완료
93 완료
94 완료
95 완료
96 완료
97 완료
98 완료
99 완료
100 완료
101 완료
102 완료
103 완료
104 완료
105 완료
106 완료
107 완료
108 완료
109 완료
110 완료
111 완료
112 완료
113 완료
114 완료
115 완료
116 완료
117 완료
118 완료
119 완료
120 완료
121 완료
122 완료
123 완료
124 완료
125 완료
126 완료
127 완료
128 완료
129 완료
130 완료
131 완료
132 완료
133 완료
134 완료
135 완료
136 완료
137 완료
138 완료
139 완료
140 완료
141 완료
142 완료
143 완료
144 완료
145 완료
146 완료
147 완료
148 완료
149 완료
150 완료
151 완료
152 완료
153 완료
154 완료
155 완료
156 완료
157 완료
158 완료
15

In [218]:
# 제품 가격 끌어오기
price=[]
for i in range(1,29):
    url = 'https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100010008&fltDispCatNo=&prdSort=01&pageIdx='+str(i)+'&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100010008_Small'
    web = urlopen(url)
    web_page = BeautifulSoup(web,'html.parser')
    price += web_page.find_all('span',{'class':'tx_cur'})
product_price=[]
for i in price:
    product_price.append(i.get_text().strip())

In [266]:
# 리뷰 수집 코드 (최종)
check = 1
product_review = []

# 제품코드 반복
for goodsNum in product_num:
    try:
        json_url = 'https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?goodsNo={}&gdasSort=05&itemNo=all_search&pageIdx=1&colData=&keywordGdasSeqs=&type=&point=&hashTag=&optionValue=&cTypeLength=0'.format(goodsNum)
        response = requests.get(json_url, headers = headers).text
        review_json = json.loads(response)
        review = []
        
        ## 나중에 Data Frame에서 리뷰가 아예없는 데이터는 삭제시키기 위해서 리뷰만 if문을 작성.
        if len(review_json['gdasList']) != 0:
            for i in range(len(review_json['gdasList'])):
                if review_json['gdasList'][i]['gdasCont'] == None or review_json['gdasList'] == None:
                    review.append(['없음'])
                    continue
                    #review.append(re.sub('[\n\r]','',review_json['gdasList'][i]['gdasCont']))
                else:
                    review.append(review_json['gdasList'][i]['gdasCont'].replace('\r','').replace('\n',''))
            product_review.append(review)

            print(check,"완료")
            check+=1
        else:
            product_review.append('없음')
            print(check,"미완성")
            check+=1
            pass
    except:
        product_review.append('없음')
        print(check,"미완성")
        check+=1
        pass


1 완료
2 완료
3 완료
4 완료
5 완료
6 완료
7 완료
8 완료
9 완료
10 완료
11 완료
12 완료
13 완료
14 완료
15 완료
16 완료
17 완료
18 완료
19 완료
20 완료
21 완료
22 완료
23 완료
24 완료
25 완료
26 완료
27 완료
28 완료
29 완료
30 완료
31 완료
32 완료
33 완료
34 완료
35 완료
36 완료
37 완료
38 완료
39 완료
40 완료
41 완료
42 완료
43 완료
44 완료
45 완료
46 완료
47 완료
48 완료
49 완료
50 완료
51 완료
52 완료
53 완료
54 완료
55 완료
56 완료
57 완료
58 완료
59 완료
60 완료
61 완료
62 완료
63 완료
64 완료
65 완료
66 완료
67 완료
68 완료
69 완료
70 완료
71 완료
72 완료
73 완료
74 완료
75 완료
76 완료
77 완료
78 완료
79 완료
80 완료
81 완료
82 완료
83 완료
84 완료
85 완료
86 완료
87 완료
88 완료
89 완료
90 완료
91 완료
92 완료
93 완료
94 완료
95 완료
96 완료
97 완료
98 완료
99 완료
100 완료
101 완료
102 완료
103 미완성
104 완료
105 완료
106 완료
107 완료
108 완료
109 완료
110 완료
111 완료
112 완료
113 완료
114 완료
115 완료
116 완료
117 완료
118 완료
119 완료
120 완료
121 완료
122 완료
123 완료
124 완료
125 완료
126 완료
127 완료
128 완료
129 완료
130 완료
131 완료
132 완료
133 완료
134 완료
135 완료
136 완료
137 완료
138 완료
139 완료
140 완료
141 완료
142 완료
143 완료
144 완료
145 완료
146 완료
147 완료
148 완료
149 완료
150 완료
151 완료
152 완료
153 완료
154 완료
155 완료
156 완료
157 완료
158 완료
1

In [267]:
len(product_review)

656

In [204]:
## 회원 피부타입 끌어오기(최종)
check = 1
masktype_set = []

# 제품코드 반복
for goodsNum in product_num:
    try:
        json_url = 'https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?goodsNo={}&gdasSort=05&itemNo=all_search&pageIdx=1&colData=&keywordGdasSeqs=&type=&point=&hashTag=&optionValue=&cTypeLength=0'.format(goodsNum)
        response = requests.get(json_url, headers = headers).text
        review_json = json.loads(response)
        masktype = []

        for i in range(len(review_json['gdasList'])):
            if review_json['gdasList'][i]['addInfoNm'] == None:
                masktype.append('')
                continue
            for j in range(len(review_json['gdasList'][i]['addInfoNm'])):
                masktype.append(review_json['gdasList'][i]['addInfoNm'][j]['mrkNm'])
        masktype_set.append(masktype)

        print(check,"완료")
        check+=1

    except:
        masktype_set += ['.']
#         #masktype_set.append('없음')
        print(check,"미완성")
        check+=1
        pass


1 완료
2 완료
3 완료
4 완료
5 완료
6 완료
7 완료
8 완료
9 완료
10 완료
11 완료
12 완료
13 완료
14 완료
15 완료
16 완료
17 완료
18 완료
19 완료
20 완료
21 완료
22 완료
23 완료
24 완료
25 완료
26 완료
27 완료
28 완료
29 완료
30 완료
31 완료
32 완료
33 완료
34 완료
35 완료
36 완료
37 완료
38 완료
39 완료
40 완료
41 완료
42 완료
43 완료
44 완료
45 완료
46 완료
47 완료
48 완료
49 완료
50 완료
51 완료
52 완료
53 완료
54 완료
55 완료
56 완료
57 완료
58 완료
59 완료
60 완료
61 완료
62 완료
63 완료
64 완료
65 완료
66 완료
67 완료
68 완료
69 완료
70 완료
71 완료
72 완료
73 완료
74 완료
75 완료
76 완료
77 완료
78 완료
79 완료
80 완료
81 완료
82 완료
83 완료
84 완료
85 완료
86 완료
87 완료
88 완료
89 완료
90 완료
91 완료
92 완료
93 완료
94 완료
95 완료
96 완료
97 완료
98 완료
99 완료
100 완료
101 완료
102 완료
103 완료
104 완료
105 완료
106 완료
107 완료
108 완료
109 완료
110 완료
111 완료
112 완료
113 완료
114 완료
115 완료
116 완료
117 완료
118 완료
119 완료
120 완료
121 완료
122 완료
123 완료
124 완료
125 완료
126 완료
127 완료
128 완료
129 완료
130 완료
131 완료
132 완료
133 완료
134 완료
135 완료
136 완료
137 완료
138 완료
139 완료
140 완료
141 완료
142 완료
143 완료
144 완료
145 완료
146 완료
147 완료
148 완료
149 완료
150 완료
151 완료
152 완료
153 완료
154 완료
155 완료
156 완료
157 완료
158 완료
15

In [272]:
## 스킨타입 (최종)
check = 1
skin_case1=[]
skin_case2=[]
skin_case3=[]

for pro in product_num:
    try:
        j_url = 'https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?goodsNo='+pro+'&gdasSort=05&itemNo=all_search&pageIdx=1&colData=&keywordGdasSeqs=&type=&point=&hashTag=&optionValue=&cTypeLength=0'
        j_data= requests.get(j_url,headers=headers).text
        reviews = json.loads(j_data)
        skin1=[]
        skin2=[]
        skin3=[]
        
        for i in range(len(reviews['gdasList'])):
            if reviews['gdasList'][i]['evalList'][0] == None:
                skin1.append('')
                skin2.append('')
                skin3.append('')
                continue
            skin1.append(reviews['gdasList'][i]['evalList'][0]['evalAnsCont'])
            skin2.append(reviews['gdasList'][i]['evalList'][1]['evalAnsCont'])
            skin3.append(reviews['gdasList'][i]['evalList'][2]['evalAnsCont'])
        skin_case1.append(skin1)
        skin_case2.append(skin2)
        skin_case3.append(skin3)
        
        print(check,"완료")
        check+=1
        
    except:
        skin_case1 += ['.']
        skin_case2 += ['.']
        skin_case3 += ['.']
        
        print(check,"미완성")
        check+=1
        


1 완료
2 완료
3 완료
4 완료
5 완료
6 완료
7 완료
8 완료
9 완료
10 완료
11 완료
12 완료
13 완료
14 완료
15 완료
16 완료
17 완료
18 완료
19 완료
20 완료
21 완료
22 완료
23 완료
24 완료
25 완료
26 완료
27 완료
28 완료
29 완료
30 완료
31 완료
32 완료
33 완료
34 완료
35 완료
36 완료
37 완료
38 미완성
39 완료
40 완료
41 완료
42 완료
43 완료
44 완료
45 미완성
46 완료
47 완료
48 완료
49 완료
50 완료
51 완료
52 완료
53 완료
54 완료
55 완료
56 완료
57 완료
58 완료
59 미완성
60 완료
61 미완성
62 완료
63 완료
64 완료
65 완료
66 완료
67 완료
68 완료
69 완료
70 완료
71 완료
72 완료
73 완료
74 완료
75 완료
76 완료
77 완료
78 완료
79 미완성
80 완료
81 완료
82 완료
83 완료
84 완료
85 완료
86 완료
87 완료
88 완료
89 완료
90 완료
91 완료
92 완료
93 완료
94 완료
95 완료
96 완료
97 완료
98 완료
99 완료
100 완료
101 완료
102 완료
103 완료
104 미완성
105 완료
106 완료
107 완료
108 완료
109 완료
110 완료
111 완료
112 완료
113 완료
114 완료
115 완료
116 완료
117 완료
118 완료
119 완료
120 완료
121 완료
122 완료
123 완료
124 완료
125 완료
126 미완성
127 완료
128 완료
129 완료
130 완료
131 완료
132 완료
133 완료
134 완료
135 완료
136 미완성
137 완료
138 완료
139 완료
140 완료
141 완료
142 완료
143 완료
144 완료
145 완료
146 완료
147 완료
148 완료
149 완료
150 완료
151 완료
152 완료
153 완료
154 완료
155 완료
156 완료
157 완료
1

In [273]:
len(skin_case1)

656

In [236]:
## 별점 (최종)
check = 1
score=[]

for pro in product_num:
    try:
        j_url = 'https://www.oliveyoung.co.kr/store/goods/getGdasNewListJson.do?goodsNo='+pro+'&gdasSort=05&itemNo=all_search&pageIdx=1&colData=&keywordGdasSeqs=&type=&point=&hashTag=&optionValue=&cTypeLength=0'
        j_data= requests.get(j_url,headers=headers).text
        reviews = json.loads(j_data)
        scr=[]
        
        for i in range(len(reviews['gdasList'])):
            scr.append(reviews['gdasList'][i]['gdasScrVal']/2)      
        score.append(scr)
        
        print(check,"완료")
        check+=1
        
    except:
        score.append('')
        
        print(check,"미완성")
        check+=1

1 완료
2 완료
3 완료
4 완료
5 완료
6 완료
7 완료
8 완료
9 완료
10 완료
11 완료
12 완료
13 완료
14 완료
15 완료
16 완료
17 완료
18 완료
19 완료
20 완료
21 완료
22 완료
23 완료
24 완료
25 완료
26 완료
27 완료
28 완료
29 완료
30 완료
31 완료
32 완료
33 완료
34 완료
35 완료
36 완료
37 완료
38 완료
39 완료
40 완료
41 완료
42 완료
43 완료
44 완료
45 완료
46 완료
47 완료
48 완료
49 완료
50 완료
51 완료
52 완료
53 완료
54 완료
55 완료
56 완료
57 완료
58 완료
59 완료
60 완료
61 완료
62 완료
63 완료
64 완료
65 완료
66 완료
67 완료
68 완료
69 완료
70 완료
71 완료
72 완료
73 완료
74 완료
75 완료
76 완료
77 완료
78 완료
79 완료
80 완료
81 완료
82 완료
83 완료
84 완료
85 완료
86 완료
87 완료
88 완료
89 완료
90 완료
91 완료
92 완료
93 완료
94 완료
95 완료
96 완료
97 완료
98 완료
99 완료
100 완료
101 완료
102 완료
103 완료
104 완료
105 완료
106 완료
107 완료
108 완료
109 완료
110 완료
111 완료
112 완료
113 완료
114 완료
115 완료
116 완료
117 완료
118 완료
119 완료
120 완료
121 완료
122 완료
123 완료
124 완료
125 완료
126 완료
127 완료
128 완료
129 완료
130 완료
131 완료
132 완료
133 완료
134 완료
135 완료
136 완료
137 완료
138 완료
139 완료
140 완료
141 완료
142 완료
143 완료
144 완료
145 완료
146 완료
147 완료
148 완료
149 완료
150 완료
151 완료
152 완료
153 완료
154 완료
155 완료
156 완료
157 완료
158 완료
15

In [237]:
len(score)

656

In [351]:
oliveyoung = pd.DataFrame({
    'num' : product_num,
    'name' : product_name_real,
    'brand': product_brand_real,
    'review' : product_review,
    'skin1' : skin_case1,
    'skin2' : skin_case2,
    'skin3' : skin_case3,
    'score' : score,
    'review_skin' : masktype_set,
    'product_total_review' : product_total_review,
    'price' : product_price,
    'url' : url_list
})

In [352]:
oliveyoung

,num,name,brand,review,skin1,skin2,skin3,score,review_skin,product_total_review,price,url
0,A000000157890,★크림스킨X노르디스크 한정기획★대용량250ml+100ml(캠핑수건or와인잔),라네즈,[■복합성 피부입니다. 평소 초가을부터 극건성으로 슬슬시동거는 타입인데요 그래서 전...,"[건성에 좋아요, 건성에 좋아요, 건성에 좋아요, 건성에 좋아요, 복합성에 좋아요,...","[주름/미백에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아...","[보통이에요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 웜톤, 각질, 다크서클, 모공, 미백, 민감성, 블랙헤드, 잡티, 주름,...","1,211","29,400원",https://www.oliveyoung.co.kr/store/goods/getGo...
1,A000000156235,★셔누P!CK★ 라운드어라운드 그린티 약산성 고보습 로션 220ml,라운드어라운드,"[😃 30대초반 남편피부타입 : 중건성(유분약간올라옴),모공적은편✔ 악건성,건성분들...","[복합성에 좋아요, 건성에 좋아요, 복합성에 좋아요, 지성에 좋아요, 건성에 좋아요...","[보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 ...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 쿨톤, 모공, 블랙헤드, 탄력, 트러블, 피지과다, 홍조, 건성, 웜톤,...",48,"19,700원",https://www.oliveyoung.co.kr/store/goods/getGo...
2,A000000141704,★된다P!CK★브링그린 티트리 시카 수딩 토너 510mL(대용량),브링그린,[원래 브링그린 사철쑥토너쓰다가 티트리토너도 인기많길레 궁금해서 구매해봤습니다 😌아...,"[복합성에 좋아요, 건성에 좋아요, 복합성에 좋아요, 복합성에 좋아요, 지성에 좋아...","[진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, ...","[보통이에요, 보통이에요, 보통이에요, 보통이에요, 보통이에요, 보통이에요, 자극없...","[4.0, 5.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, ...","[복합성, 웜톤, 각질, 모공, 미백, 민감성, 블랙헤드, 잡티, 주름, 탄력, 트...","9,943","16,720원",https://www.oliveyoung.co.kr/store/goods/getGo...
3,A000000016686,★한정기획★아벤느 트릭세라 로션 200ml #윤기보습 #윤광로션 (100ml 증정),아벤느,[아벤느 트릭세라 로션 사용중인데요 원래 다른 화장품사면서 올리브영에서 샘플로 받아...,"[복합성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 건성에 좋아요, 건성에 좋아요...","[보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극이 느껴져요, 자극없이 순해요, 자극없이 ...","[5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 웜톤, 모공, 미백, 민감성, 블랙헤드, 잡티, 탄력, 복합성, 웜톤, ...","1,579","23,100원",https://www.oliveyoung.co.kr/store/goods/getGo...
4,A000000137180,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),라운드랩,[제가 피부가 많이 예민한 편이라서 기초제품에 오랫동안 정착을 못 하는 편인데 독도...,"[복합성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 건성에 좋아요, 복합성에 좋아...","[진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 보습에 좋아요, 진정에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 ...","[5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[지성, 웜톤, 모공, 민감성, 민감성, 봄웜톤, 각질, 모공, 미백, 민감성, 트...","4,854","27,000원",https://www.oliveyoung.co.kr/store/goods/getGo...
5,A000000148569,아이디얼포맨 퍼펙트올인원 150mL,아이디얼포맨,[💚남성스킨케어 1등 아이디얼포맨과 제모용품 1등 질레트의 만남이라니..💚스킨케어제...,"[복합성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 복합성에 좋아요, 복합성에 좋...","[보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 주름/미백에 좋아...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 보통이에요...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...","[복합성, 웜톤, 각질, 모공, 미백, 민감성, 블랙헤드, 잡티, 주름, 탄력, 트...","4,313","20,900원",https://www.oliveyoung.co.kr/store/goods/getGo...
6,A000000155253,아비브 어성초 카밍 토너 스킨부스터 더블 기획,아비브,[💟후기💟저는 아비브 찐팬이고 아비브 제품도 정말 여러가지 사용해봤는데 정말 그 중...,"[건성에 좋아요, 건성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 지성에 좋아요,...","[보습에 좋아요, 보습에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 ...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","[복합성, 가을웜톤, 모공, 민감성, 아토피, 잡티, 트러블, 복합성, 가을웜톤, ...","1,519","27,300원",https://www.oliveyoung.co.kr/store/goods/getGo...
7,A000000143827,닥터자르트 시카페어 토너(250mL+화장솜 20매입) 세트,닥터자르트,[시카페어 토너 진짜 짱임아니! 시카페어 라인 다 짱임!!!맞는 기초화장품 찾기 어...,"[복합성에 좋아요, 복합성에 좋아요, 복합성에 좋아요, 복합성에 좋아요, 복합성에 ...","[진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 보통이에요...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 웜톤, 각질, 모공, 민감성, 잡티, 트러블, 지성, 봄웜톤, 블랙헤드,...","1,031","24,500원",https://www.oliveyoung.co.kr/store/goods/getGo...
8,A000000155120,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),더랩바이블랑두,"[큰용량, 작은용량 할거없이 적어도 5-6통정도 비우고 또 재구매하는 리뷰입니다 일...","[복합성에 좋아요, 복합성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 건성에 좋아...","[보습에 좋아요, 보습에 좋아요, 진정에 좋아요, 보습에 좋아요, 보습에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 봄웜톤, 미백, 블랙헤드, 복합성, 쿨톤, 모공, 잡티, 피지과다, 홍조...","1,266","16,800원",https://www.oliveyoung.co.kr/

In [353]:
## df에서 리뷰가 1개도 없는 없음 항목 지우기
idx = oliveyoung[oliveyoung['review'] == '없음'].index
oliveyoung = oliveyoung.drop(idx)

In [354]:
oliveyoung

,num,name,brand,review,skin1,skin2,skin3,score,review_skin,product_total_review,price,url
0,A000000157890,★크림스킨X노르디스크 한정기획★대용량250ml+100ml(캠핑수건or와인잔),라네즈,[■복합성 피부입니다. 평소 초가을부터 극건성으로 슬슬시동거는 타입인데요 그래서 전...,"[건성에 좋아요, 건성에 좋아요, 건성에 좋아요, 건성에 좋아요, 복합성에 좋아요,...","[주름/미백에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아...","[보통이에요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 웜톤, 각질, 다크서클, 모공, 미백, 민감성, 블랙헤드, 잡티, 주름,...","1,211","29,400원",https://www.oliveyoung.co.kr/store/goods/getGo...
1,A000000156235,★셔누P!CK★ 라운드어라운드 그린티 약산성 고보습 로션 220ml,라운드어라운드,"[😃 30대초반 남편피부타입 : 중건성(유분약간올라옴),모공적은편✔ 악건성,건성분들...","[복합성에 좋아요, 건성에 좋아요, 복합성에 좋아요, 지성에 좋아요, 건성에 좋아요...","[보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 ...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 쿨톤, 모공, 블랙헤드, 탄력, 트러블, 피지과다, 홍조, 건성, 웜톤,...",48,"19,700원",https://www.oliveyoung.co.kr/store/goods/getGo...
2,A000000141704,★된다P!CK★브링그린 티트리 시카 수딩 토너 510mL(대용량),브링그린,[원래 브링그린 사철쑥토너쓰다가 티트리토너도 인기많길레 궁금해서 구매해봤습니다 😌아...,"[복합성에 좋아요, 건성에 좋아요, 복합성에 좋아요, 복합성에 좋아요, 지성에 좋아...","[진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, ...","[보통이에요, 보통이에요, 보통이에요, 보통이에요, 보통이에요, 보통이에요, 자극없...","[4.0, 5.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, ...","[복합성, 웜톤, 각질, 모공, 미백, 민감성, 블랙헤드, 잡티, 주름, 탄력, 트...","9,943","16,720원",https://www.oliveyoung.co.kr/store/goods/getGo...
3,A000000016686,★한정기획★아벤느 트릭세라 로션 200ml #윤기보습 #윤광로션 (100ml 증정),아벤느,[아벤느 트릭세라 로션 사용중인데요 원래 다른 화장품사면서 올리브영에서 샘플로 받아...,"[복합성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 건성에 좋아요, 건성에 좋아요...","[보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극이 느껴져요, 자극없이 순해요, 자극없이 ...","[5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 웜톤, 모공, 미백, 민감성, 블랙헤드, 잡티, 탄력, 복합성, 웜톤, ...","1,579","23,100원",https://www.oliveyoung.co.kr/store/goods/getGo...
4,A000000137180,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),라운드랩,[제가 피부가 많이 예민한 편이라서 기초제품에 오랫동안 정착을 못 하는 편인데 독도...,"[복합성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 건성에 좋아요, 복합성에 좋아...","[진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 보습에 좋아요, 진정에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 ...","[5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[지성, 웜톤, 모공, 민감성, 민감성, 봄웜톤, 각질, 모공, 미백, 민감성, 트...","4,854","27,000원",https://www.oliveyoung.co.kr/store/goods/getGo...
5,A000000148569,아이디얼포맨 퍼펙트올인원 150mL,아이디얼포맨,[💚남성스킨케어 1등 아이디얼포맨과 제모용품 1등 질레트의 만남이라니..💚스킨케어제...,"[복합성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 복합성에 좋아요, 복합성에 좋...","[보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 보습에 좋아요, 주름/미백에 좋아...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 보통이에요...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...","[복합성, 웜톤, 각질, 모공, 미백, 민감성, 블랙헤드, 잡티, 주름, 탄력, 트...","4,313","20,900원",https://www.oliveyoung.co.kr/store/goods/getGo...
6,A000000155253,아비브 어성초 카밍 토너 스킨부스터 더블 기획,아비브,[💟후기💟저는 아비브 찐팬이고 아비브 제품도 정말 여러가지 사용해봤는데 정말 그 중...,"[건성에 좋아요, 건성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 지성에 좋아요,...","[보습에 좋아요, 보습에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 ...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","[복합성, 가을웜톤, 모공, 민감성, 아토피, 잡티, 트러블, 복합성, 가을웜톤, ...","1,519","27,300원",https://www.oliveyoung.co.kr/store/goods/getGo...
7,A000000143827,닥터자르트 시카페어 토너(250mL+화장솜 20매입) 세트,닥터자르트,[시카페어 토너 진짜 짱임아니! 시카페어 라인 다 짱임!!!맞는 기초화장품 찾기 어...,"[복합성에 좋아요, 복합성에 좋아요, 복합성에 좋아요, 복합성에 좋아요, 복합성에 ...","[진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, 진정에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 보통이에요...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 웜톤, 각질, 모공, 민감성, 잡티, 트러블, 지성, 봄웜톤, 블랙헤드,...","1,031","24,500원",https://www.oliveyoung.co.kr/store/goods/getGo...
8,A000000155120,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),더랩바이블랑두,"[큰용량, 작은용량 할거없이 적어도 5-6통정도 비우고 또 재구매하는 리뷰입니다 일...","[복합성에 좋아요, 복합성에 좋아요, 복합성에 좋아요, 건성에 좋아요, 건성에 좋아...","[보습에 좋아요, 보습에 좋아요, 진정에 좋아요, 보습에 좋아요, 보습에 좋아요, ...","[자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 순해요, 자극없이 ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[복합성, 봄웜톤, 미백, 블랙헤드, 복합성, 쿨톤, 모공, 잡티, 피지과다, 홍조...","1,266","16,800원",https://www.oliveyoung.co.kr/

In [358]:
oliveyoung.to_excel('oliveyoung_df1.xlsx', encoding='euc-kr')

# 리뷰에서 이모티콘 지우기

In [359]:
## DF INDEX 정렬된 엑셀로 다시 불러오는 이유는 아래 이모티콘 제거코드 적용시 list 타입이 아닌 str 타입으로 적용하기 위함
oliveyoung = pd.read_excel('oliveyoung_df1.xlsx')

In [360]:
del oliveyoung['Unnamed: 0']
oliveyoung

,num,name,brand,review,skin1,skin2,skin3,score,review_skin,product_total_review,price,url
0,A000000157890,★크림스킨X노르디스크 한정기획★대용량250ml+100ml(캠핑수건or와인잔),라네즈,['■복합성 피부입니다. 평소 초가을부터 극건성으로 슬슬시동거는 타입인데요 그래서 ...,"['건성에 좋아요', '건성에 좋아요', '건성에 좋아요', '건성에 좋아요', '...","['주름/미백에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요'...","['보통이에요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요', ...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '웜톤', '각질', '다크서클', '모공', '미백', '민감성',...","1,211","29,400원",https://www.oliveyoung.co.kr/store/goods/getGo...
1,A000000156235,★셔누P!CK★ 라운드어라운드 그린티 약산성 고보습 로션 220ml,라운드어라운드,"['😃 30대초반 남편피부타입 : 중건성(유분약간올라옴),모공적은편✔ 악건성,건성분...","['복합성에 좋아요', '건성에 좋아요', '복합성에 좋아요', '지성에 좋아요',...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '쿨톤', '모공', '블랙헤드', '탄력', '트러블', '피지과다...",48,"19,700원",https://www.oliveyoung.co.kr/store/goods/getGo...
2,A000000141704,★된다P!CK★브링그린 티트리 시카 수딩 토너 510mL(대용량),브링그린,['원래 브링그린 사철쑥토너쓰다가 티트리토너도 인기많길레 궁금해서 구매해봤습니다 😌...,"['복합성에 좋아요', '건성에 좋아요', '복합성에 좋아요', '복합성에 좋아요'...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['보통이에요', '보통이에요', '보통이에요', '보통이에요', '보통이에요', ...","[4.0, 5.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, ...","['복합성', '웜톤', '각질', '모공', '미백', '민감성', '블랙헤드',...","9,943","16,720원",https://www.oliveyoung.co.kr/store/goods/getGo...
3,A000000016686,★한정기획★아벤느 트릭세라 로션 200ml #윤기보습 #윤광로션 (100ml 증정),아벤느,['아벤느 트릭세라 로션 사용중인데요 원래 다른 화장품사면서 올리브영에서 샘플로 받...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '건성에 좋아요',...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극이 느껴져요', '자극없이 순해요...","[5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '웜톤', '모공', '미백', '민감성', '블랙헤드', '잡티',...","1,579","23,100원",https://www.oliveyoung.co.kr/store/goods/getGo...
4,A000000137180,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),라운드랩,['제가 피부가 많이 예민한 편이라서 기초제품에 오랫동안 정착을 못 하는 편인데 독...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '건성에 좋아요',...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['지성', '웜톤', '모공', '민감성', '민감성', '봄웜톤', '각질', ...","4,854","27,000원",https://www.oliveyoung.co.kr/store/goods/getGo...
5,A000000148569,아이디얼포맨 퍼펙트올인원 150mL,아이디얼포맨,['💚남성스킨케어 1등 아이디얼포맨과 제모용품 1등 질레트의 만남이라니..💚스킨케어...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '복합성에 좋아요'...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...","['복합성', '웜톤', '각질', '모공', '미백', '민감성', '블랙헤드',...","4,313","20,900원",https://www.oliveyoung.co.kr/store/goods/getGo...
6,A000000155253,아비브 어성초 카밍 토너 스킨부스터 더블 기획,아비브,"[""💟후기💟저는 아비브 찐팬이고 아비브 제품도 정말 여러가지 사용해봤는데 정말 그 ...","['건성에 좋아요', '건성에 좋아요', '복합성에 좋아요', '건성에 좋아요', ...","['보습에 좋아요', '보습에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","['복합성', '가을웜톤', '모공', '민감성', '아토피', '잡티', '트러블...","1,519","27,300원",https://www.oliveyoung.co.kr/store/goods/getGo...
7,A000000143827,닥터자르트 시카페어 토너(250mL+화장솜 20매입) 세트,닥터자르트,['시카페어 토너 진짜 짱임아니! 시카페어 라인 다 짱임!!!맞는 기초화장품 찾기 ...,"['복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '웜톤', '각질', '모공', '민감성', '잡티', '트러블', ...","1,031","24,500원",https://www.oliveyoung.co.kr/store/goods/getGo...
8,A000000155120,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),더랩바이블랑두,"['큰용량, 작은용량 할거없이 적어도 5-6통정도 비우고 또 재구매하는 리뷰입니다 ...","['복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요'...","['보습에 좋아요', '보습에 좋아요', '진정에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '봄웜톤', '미백', '블랙헤드', '복합성', '쿨톤', '모공'...","1,266","16,800원",https://www.oliveyoung.co.

In [361]:
## 이모티콘 값만 제거하는 함수 선언
def rmEmoji(inputData):
    return inputData.encode('euc-kr', 'ignore').decode('euc-kr')

In [363]:
## 이모티콘 값을 무시하고 텍스트 데이터만 추출하여 저장
for i in range(len(oliveyoung['review'])):
    oliveyoung['review'][i] = rmEmoji(oliveyoung['review'][i])

In [365]:
# 최종 DF 완성
oliveyoung.to_excel('oliveyoung_df.xlsx', encoding='euc-kr', index = False)

In [366]:
oliveyoung

,num,name,brand,review,skin1,skin2,skin3,score,review_skin,product_total_review,price,url
0,A000000157890,★크림스킨X노르디스크 한정기획★대용량250ml+100ml(캠핑수건or와인잔),라네즈,['■복합성 피부입니다. 평소 초가을부터 극건성으로 슬슬시동거는 타입인데요 그래서 ...,"['건성에 좋아요', '건성에 좋아요', '건성에 좋아요', '건성에 좋아요', '...","['주름/미백에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요'...","['보통이에요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요', ...","[5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '웜톤', '각질', '다크서클', '모공', '미백', '민감성',...","1,211","29,400원",https://www.oliveyoung.co.kr/store/goods/getGo...
1,A000000156235,★셔누P!CK★ 라운드어라운드 그린티 약산성 고보습 로션 220ml,라운드어라운드,"[' 30대초반 남편피부타입 : 중건성(유분약간올라옴),모공적은편 악건성,건성분들은...","['복합성에 좋아요', '건성에 좋아요', '복합성에 좋아요', '지성에 좋아요',...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '쿨톤', '모공', '블랙헤드', '탄력', '트러블', '피지과다...",48,"19,700원",https://www.oliveyoung.co.kr/store/goods/getGo...
2,A000000141704,★된다P!CK★브링그린 티트리 시카 수딩 토너 510mL(대용량),브링그린,['원래 브링그린 사철쑥토너쓰다가 티트리토너도 인기많길레 궁금해서 구매해봤습니다 아...,"['복합성에 좋아요', '건성에 좋아요', '복합성에 좋아요', '복합성에 좋아요'...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['보통이에요', '보통이에요', '보통이에요', '보통이에요', '보통이에요', ...","[4.0, 5.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, ...","['복합성', '웜톤', '각질', '모공', '미백', '민감성', '블랙헤드',...","9,943","16,720원",https://www.oliveyoung.co.kr/store/goods/getGo...
3,A000000016686,★한정기획★아벤느 트릭세라 로션 200ml #윤기보습 #윤광로션 (100ml 증정),아벤느,['아벤느 트릭세라 로션 사용중인데요 원래 다른 화장품사면서 올리브영에서 샘플로 받...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '건성에 좋아요',...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극이 느껴져요', '자극없이 순해요...","[5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '웜톤', '모공', '미백', '민감성', '블랙헤드', '잡티',...","1,579","23,100원",https://www.oliveyoung.co.kr/store/goods/getGo...
4,A000000137180,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),라운드랩,['제가 피부가 많이 예민한 편이라서 기초제품에 오랫동안 정착을 못 하는 편인데 독...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '건성에 좋아요',...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['지성', '웜톤', '모공', '민감성', '민감성', '봄웜톤', '각질', ...","4,854","27,000원",https://www.oliveyoung.co.kr/store/goods/getGo...
5,A000000148569,아이디얼포맨 퍼펙트올인원 150mL,아이디얼포맨,['남성스킨케어 1등 아이디얼포맨과 제모용품 1등 질레트의 만남이라니..스킨케어제품...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '복합성에 좋아요'...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...","['복합성', '웜톤', '각질', '모공', '미백', '민감성', '블랙헤드',...","4,313","20,900원",https://www.oliveyoung.co.kr/store/goods/getGo...
6,A000000155253,아비브 어성초 카밍 토너 스킨부스터 더블 기획,아비브,"[""후기저는 아비브 찐팬이고 아비브 제품도 정말 여러가지 사용해봤는데 정말 그 중에...","['건성에 좋아요', '건성에 좋아요', '복합성에 좋아요', '건성에 좋아요', ...","['보습에 좋아요', '보습에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","['복합성', '가을웜톤', '모공', '민감성', '아토피', '잡티', '트러블...","1,519","27,300원",https://www.oliveyoung.co.kr/store/goods/getGo...
7,A000000143827,닥터자르트 시카페어 토너(250mL+화장솜 20매입) 세트,닥터자르트,['시카페어 토너 진짜 짱임아니! 시카페어 라인 다 짱임!!!맞는 기초화장품 찾기 ...,"['복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '웜톤', '각질', '모공', '민감성', '잡티', '트러블', ...","1,031","24,500원",https://www.oliveyoung.co.kr/store/goods/getGo...
8,A000000155120,더랩바이블랑두 올리고 히알루론산 5000토너 200ml 기획세트 (토너100ml 증정),더랩바이블랑두,"['큰용량, 작은용량 할거없이 적어도 5-6통정도 비우고 또 재구매하는 리뷰입니다 ...","['복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요'...","['보습에 좋아요', '보습에 좋아요', '진정에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","['복합성', '봄웜톤', '미백', '블랙헤드', '복합성', '쿨톤', '모공'...","1,266","16,800원",https://www.oliveyoung.co.

## 1차 데이터 수집 완료

In [400]:
oliveyoung = pd.read_excel('oliveyoung_df.xlsx')

# 2차 데이터 수집 시작

# 문자열 분석 시작

In [403]:
import nltk
from konlpy.tag import Okt
from konlpy.tag import Twitter

In [404]:
vectorizer = TfidfVectorizer()

In [405]:
twitter = Twitter()

In [294]:
from sklearn.feature_extraction.text import CountVectorizer  # 피체 벡터화
from sklearn.metrics.pairwise import cosine_similarity  # 코사인 유사도

# 피부 코사인 유사도 분석

In [406]:
skin1_list=[]
for i in range(len(oliveyoung['skin1'])):
    nouns = twitter.nouns(oliveyoung['skin1'][i])
    nouns = ' '.join(nouns)
    skin1_list.append(nouns)

In [407]:
oliveyoung['skin1_noun'] = skin1_list

In [409]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
skin1 = count_vect_category.fit_transform(oliveyoung['skin1_noun']) 
skin1_cosine = cosine_similarity(skin1,skin1)
skin1_cosine_ind = skin1_cosine.argsort()[:, ::-1]

In [410]:
skin2_list=[]
for i in range(len(oliveyoung['skin2'])):
    nouns = twitter.nouns(oliveyoung['skin2'][i])
    nouns = ' '.join(nouns)
    skin2_list.append(nouns)

In [411]:
oliveyoung['skin2_noun'] = skin2_list

In [412]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
skin2 = count_vect_category.fit_transform(oliveyoung['skin2_noun']) 
skin2_cosine = cosine_similarity(skin2,skin2)

In [413]:
oliveyoung['skin3_noun'] = oliveyoung['skin3']

In [414]:
count_vect_category = CountVectorizer(min_df=0, ngram_range=(1,2))
skin3 = count_vect_category.fit_transform(oliveyoung['skin3'])
skin3_cosine = cosine_similarity(skin3,skin3)
skin3_cosine

array([[1.        , 0.99674572, 0.57745503, ..., 0.97740908, 0.97847381,
        0.99695659],
       [0.99674572, 1.        , 0.53489604, ..., 0.96486073, 0.97339949,
        0.99327782],
       [0.57745503, 0.53489604, 1.        , ..., 0.63883745, 0.5864868 ,
        0.58536997],
       ...,
       [0.97740908, 0.96486073, 0.63883745, ..., 1.        , 0.98241699,
        0.9887907 ],
       [0.97847381, 0.97339949, 0.5864868 , ..., 0.98241699, 1.        ,
        0.98589678],
       [0.99695659, 0.99327782, 0.58536997, ..., 0.9887907 , 0.98589678,
        1.        ]])

## 제품 피부 리뷰 빈도수 체크하기

In [416]:
# 스킨 1 정렬하기
from collections import Counter
word_dic = {}
for i in range(len(oliveyoung['skin1_noun'])):
    result = Counter(oliveyoung['skin1_noun'][i].split())
    word_dic = dict(result)
    word_dic = sorted(word_dic.items(), key=lambda x: x[1], reverse=True)
    oliveyoung['skin1_noun'][i] = word_dic
oliveyoung['skin1_noun']

0               [(건성, 9), (복합, 1)]
1      [(건성, 5), (복합, 4), (지성, 1)]
2      [(복합, 5), (지성, 4), (건성, 1)]
3               [(건성, 6), (복합, 4)]
4               [(복합, 6), (건성, 4)]
5               [(복합, 8), (건성, 2)]
6      [(복합, 5), (건성, 4), (지성, 1)]
7               [(복합, 8), (건성, 2)]
8               [(복합, 8), (건성, 2)]
9               [(건성, 7), (복합, 3)]
10              [(건성, 6), (복합, 4)]
11     [(복합, 7), (지성, 2), (건성, 1)]
12              [(복합, 8), (건성, 2)]
13     [(복합, 5), (건성, 4), (지성, 1)]
14     [(복합, 7), (지성, 2), (건성, 1)]
15     [(복합, 8), (지성, 1), (건성, 1)]
16     [(복합, 4), (지성, 4), (건성, 2)]
17     [(복합, 4), (건성, 2), (지성, 1)]
18     [(건성, 8), (지성, 1), (복합, 1)]
19              [(복합, 7), (건성, 3)]
20              [(복합, 6), (건성, 4)]
21              [(복합, 9), (건성, 1)]
22     [(복합, 5), (지성, 4), (건성, 1)]
23     [(지성, 4), (복합, 3), (건성, 3)]
24     [(복합, 6), (지성, 2), (건성, 2)]
25     [(건성, 5), (지성, 4), (복합, 1)]
26     [(복합, 6), (건성, 3), (지성, 1)]
27     [(복합, 5), (건성, 4), (지성, 1)]
28     [(지성, 5), (복합

In [417]:
# 스킨 2 정렬하기
from collections import Counter
word_dic = {}
for i in range(len(oliveyoung['skin2_noun'])):
    result = Counter(oliveyoung['skin2_noun'][i].split())
    word_dic = dict(result)
    word_dic = sorted(word_dic.items(), key=lambda x: x[1], reverse=True)
    oliveyoung['skin2_noun'][i] = word_dic
oliveyoung['skin2_noun']

0               [(보습, 9), (주름, 1), (미백, 1)]
1                                [(보습, 10)]
2                                [(진정, 10)]
3                                [(보습, 10)]
4                        [(진정, 5), (보습, 5)]
5               [(보습, 7), (주름, 3), (미백, 3)]
6                        [(진정, 7), (보습, 3)]
7                                [(진정, 10)]
8                        [(보습, 7), (진정, 3)]
9                        [(보습, 8), (진정, 2)]
10                       [(보습, 9), (진정, 1)]
11                       [(진정, 5), (보습, 5)]
12                               [(진정, 10)]
13                       [(진정, 9), (보습, 1)]
14     [(진정, 7), (보습, 2), (주름, 1), (미백, 1)]
15     [(주름, 7), (미백, 7), (진정, 2), (보습, 1)]
16                       [(보습, 7), (진정, 3)]
17                       [(보습, 6), (진정, 1)]
18                               [(보습, 10)]
19                       [(보습, 9), (진정, 1)]
20                       [(진정, 7), (보습, 3)]
21                       [(보습, 6), (진정, 4)]
22                       [(보습, 6

In [320]:
import sys

In [418]:
# 스킨 3 정렬하기
word_dic = {}
for i in range(len(oliveyoung['skin3'])):
    result = Counter(oliveyoung['skin3'][i].replace(' ','').replace('[','').replace(']','').split(','))
    word_dic = dict(result)
    word_dic = sorted(word_dic.items(), key=lambda x: x[1], reverse=True)
    oliveyoung['skin3_noun'][i] = word_dic
oliveyoung['skin3_noun']

0                      [('자극없이순해요', 9), ('보통이에요', 1)]
1                                   [('자극없이순해요', 10)]
2                      [('보통이에요', 7), ('자극없이순해요', 3)]
3                    [('자극없이순해요', 9), ('자극이느껴져요', 1)]
4                                   [('자극없이순해요', 10)]
5                      [('자극없이순해요', 7), ('보통이에요', 3)]
6                                   [('자극없이순해요', 10)]
7                      [('자극없이순해요', 8), ('보통이에요', 2)]
8                      [('자극없이순해요', 9), ('보통이에요', 1)]
9                      [('자극없이순해요', 9), ('보통이에요', 1)]
10                                  [('자극없이순해요', 10)]
11                     [('자극없이순해요', 9), ('보통이에요', 1)]
12                                  [('자극없이순해요', 10)]
13                                  [('자극없이순해요', 10)]
14     [('자극없이순해요', 7), ('보통이에요', 2), ('자극이느껴져요', 1)]
15                     [('자극이느껴져요', 6), ('보통이에요', 4)]
16                     [('자극없이순해요', 9), ('보통이에요', 1)]
17                                   [('자극없이순해요', 7)]
18                     [('자극

In [419]:
oliveyoung.to_excel('oliveyoung_skin_count.xlsx', encoding='euc-kr',columns=['num','skin1_noun','skin2_noun','skin3_noun'])

In [142]:
print(a)

None


In [119]:
olive_df

,num,name,brand,review,skin1,skin2,skin3,score,review_skin,review_noun,skin1_noun,skin2_noun,score_mean,skin3_noun
0,A000000157890,★크림스킨X노르디스크 한정기획★대용량250ml+100ml(캠핑수건or와인잔),라네즈,['노르디스크 캠핑용 수건 기획세트 출시 기념으로 세일하길래 마침 쓰던 토너도 다 ...,"['건성에 좋아요', '건성에 좋아요', '건성에 좋아요', '복합성에 좋아요', ...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[['지성', '웜톤', '모공', '블랙헤드', '트러블', '피지과다'], ['...",노르 디스크 캠핑 용 수건 기획 세트 출시 기념 세 마침 토너 얼른 구매 일해 저 ...,"{'건성': 8, '복합': 2}",{'보습': 10},4.9,"{'['자극없이순해요'': 1, ''자극없이순해요'': 8, ''자극없이순해요']'..."
1,A000000141704,★된다P!CK★브링그린 티트리 시카 수딩 토너 510mL(대용량),브링그린,['원래 브링그린 사철쑥토너쓰다가 티트리토너도 인기많길레 궁금해서 구매해봤습니다 아...,"['복합성에 좋아요', '건성에 좋아요', '복합성에 좋아요', '복합성에 좋아요'...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['보통이에요', '보통이에요', '보통이에요', '보통이에요', '보통이에요', ...","[4.0, 5.0, 4.0, 4.0, 3.0, 5.0, 5.0, 4.0, 5.0, ...","[['복합성', '웜톤', '각질', '모공', '미백', '민감성', '블랙헤드'...",원래 브링 그린 사철 쑥 토너 티 트리 토너 인기 레 구매 사용 한지 정도 피부 요...,"{'복합': 5, '건성': 1, '지성': 4}",{'진정': 10},4.3,"{'['보통이에요'': 1, ''보통이에요'': 5, ''자극없이순해요'': 3, ..."
2,A000000143827,닥터자르트 시카페어 토너(250mL+화장솜 20매입) 세트,닥터자르트,['시카페어 토너 진짜 짱임아니! 시카페어 라인 다 짱임!!!맞는 기초화장품 찾기 ...,"['복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, ...","[['복합성', '웜톤', '각질', '모공', '민감성', '잡티', '트러블']...",카페 토너 진짜 카페 라인 임 기초 화장품 찾기 카페 라인 보고 이후 계속 중 세안...,"{'복합': 8, '건성': 2}",{'진정': 10},4.9,"{'['자극없이순해요'': 1, ''자극없이순해요'': 7, ''보통이에요'': 1..."
3,A000000155253,아비브 어성초 카밍 토너 스킨부스터 더블 기획,아비브,"[""후기저는 아비브 찐팬이고 아비브 제품도 정말 여러가지 사용해봤는데 정말 그 중에...","['건성에 좋아요', '건성에 좋아요', '복합성에 좋아요', '건성에 좋아요', ...","['보습에 좋아요', '보습에 좋아요', '진정에 좋아요', '진정에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, 5.0, ...","[['복합성', '가을웜톤', '모공', '민감성', '아토피', '잡티', '트러...",후기 저 아비 브 팬 아비 브 제품 정말 여러가지 사용 정말 그 중 최고 수 제품 ...,"{'건성': 4, '복합': 5, '지성': 1}","{'보습': 3, '진정': 7}",4.8,"{'['자극없이순해요'': 1, ''자극없이순해요'': 8, ''자극없이순해요']'..."
4,A000000016686,★한정기획★아벤느 트릭세라 로션 200ml #윤기보습 #윤광로션 (100ml 증정),아벤느,['아벤느 트릭세라 로션 사용중인데요 원래 다른 화장품사면서 올리브영에서 샘플로 받...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '건성에 좋아요',...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극이 느껴져요', '자극없이 순해요...","[5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[['복합성', '웜톤', '모공', '미백', '민감성', '블랙헤드', '잡티'...",아벤느 트릭 세라 로션 사용 요 원래 다른 화장품 사면 올리브영 샘플 미니 이번 행...,"{'복합': 4, '건성': 6}",{'보습': 10},4.8,"{'['자극없이순해요'': 1, ''자극없이순해요'': 7, ''자극이느껴져요'':..."
5,A000000148569,아이디얼포맨 퍼펙트올인원 150mL,아이디얼포맨,['남성스킨케어 1등 아이디얼포맨과 제모용품 1등 질레트의 만남이라니..스킨케어제품...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '복합성에 좋아요'...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 4.0, 5.0, ...","[['복합성', '웜톤', '각질', '모공', '미백', '민감성', '블랙헤드'...",남성 스킨 케어 아이디얼 포맨 용품 질레트 만남 스킨 케어 제품 남편 정착 아이디얼...,"{'복합': 8, '건성': 2}","{'보습': 7, '주름': 3, '미백': 3}",4.9,"{'['자극없이순해요'': 1, ''자극없이순해요'': 6, ''보통이에요'': 2..."
6,A000000155000,하다라보 히아풀베리어 피에이치 밸런싱 토너 기획(+토너20ml+클렌징젤15g증정),하다라보,['하다라보 보습이 워낙 좋다길래 친구 살때 같이 사봤어요 ㅎ 엄청 촉촉하고 물같은...,"['건성에 좋아요', '복합성에 좋아요', '복합성에 좋아요', '복합성에 좋아요'...","['보습에 좋아요', '보습에 좋아요', '보습에 좋아요', '진정에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 5.0, 5.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[['건성', '웜톤', '잡티', '주름', '탄력'], ['복합성', '쿨톤',...",라보 보습 워낙 친구 살때 물 형 화장 솜 얼굴 두 팩 거 처럼 느낌 여름 지금 쓰...,"{'건성': 5, '복합': 5}","{'보습': 8, '진정': 2}",4.9,"{'['자극없이순해요'': 1, ''자극없이순해요'': 8, ''자극없이순해요']'..."
7,A000000137180,라운드랩 1025 독도 토너 대용량 500ml 기획(토너 100ml 증정),라운드랩,['제가 피부가 많이 예민한 편이라서 기초제품에 오랫동안 정착을 못 하는 편인데 독...,"['복합성에 좋아요', '복합성에 좋아요', '건성에 좋아요', '건성에 좋아요',...","['진정에 좋아요', '진정에 좋아요', '진정에 좋아요', '보습에 좋아요', '...","['자극없이 순해요', '자극없이 순해요', '자극없이 순해요', '자극없이 순해요...","[5.0, 3.0, 5.0, 5.0,

## 데이터 수집 최종완료